In [ ]:
# IMPORT PACKAGES FOR PROJECT

# !pip install wordcloud, pandas, matplotlib, PIL
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [ ]:
# Define Function to calculate the number of days in a year
def days_in_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return 366
            else:
                return 365
        else:
            return 366
    else:
        return 365

In [ ]:
# Define function to extract text befor delimiter
def extract_delim(names):
    results = []
    delimiters = {':', ';', '!'}
    for name in names:
        for delimiter in delimiters:
            if delimiter in name:
                result = name.split(delimiter, 1)[0].strip()
                results.append(result)
                break
        else:
            results.append(name)
    return results

In [ ]:
# define function to get book summary
def get_book_summaries(book_titles):
    summaries = []
    for book_title in book_titles:
        try:
            # Amazon
            amazon_url = f"https://www.amazon.com/s?k={book_title}"
            response = requests.get(amazon_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            search_results = soup.find_all('div', {'data-component-type': 's-search-result'})
            if len(search_results) == 0:
                amazon_summary = f"No results found for '{book_title}' on Amazon."
            else:
                top_result = search_results[0]
                book_url = "https://www.amazon.com" + top_result.find('a', {'class': 'a-link-normal'})['href']
                response = requests.get(book_url)
                soup = BeautifulSoup(response.content, 'html.parser')
                amazon_summary = soup.find('div', {'id': 'bookDescription_feature_div'}).find('noscript').get_text().strip()
        except:
            amazon_summary = f"Error getting summary for '{book_title}' on Amazon."

        try:
            # Wikipedia
            wikipedia_url = f"https://en.wikipedia.org/wiki/{book_title.replace(' ', '_')}"
            response = requests.get(wikipedia_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            wikipedia_summary = soup.find('div', {'class': 'mw-parser-output'}).find_all('p')[0].get_text().strip()
        except:
            wikipedia_summary = f"Error getting summary for '{book_title}' on Wikipedia."

        try:
            # Goodreads
            goodreads_url = f"https://www.goodreads.com/search?q={book_title}"
            response = requests.get(goodreads_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            search_results = soup.find_all('tr', {'itemtype': 'http://schema.org/Book'})
            if len(search_results) == 0:
                goodreads_summary = f"No results found for '{book_title}' on Goodreads."
            else:
                top_result = search_results[0]
                book_url = top_result.find('a', {'class': 'bookTitle'})['href']
                response = requests.get(book_url)
                soup = BeautifulSoup(response.content, 'html.parser')
                goodreads_summary = soup.find('div', {'id': 'description'}).find_all('span')[1].get_text().strip()
        except:
            goodreads_summary = f"Error getting summary for '{book_title}' on Goodreads."

        summaries.append({
            'book_title': book_title,
            'Amazon': amazon_summary,
            'Wikipedia': wikipedia_summary,
            'Goodreads': goodreads_summary
        })

    return summaries

In [ ]:
book_list2021 = {1:{'Name':"The richest man in Babylon", 'Author':"George S. Clason", 'Pages':224, 'Format': "Print"}
                 , 2:{'Name':"The slight edge", 'Author':"Jeff Olson", 'Pages':280, 'Format': "Print"}
                 , 3:{'Name':"Intermittent fasting basics", 'Author':"Lindsay Boyers", 'Pages':219, 'Format': "Print"}
                 , 4:{'Name':"The one thing", 'Author':"Gary Keller", 'Pages':240, 'Format': "Print"}
                 , 5:{'Name':"Who moved my cheese", 'Author':"Spencer Johnson", 'Pages':94, 'Format': "Print"}
                 , 6:{'Name':"Think and grow rich", 'Author':"Napoleon Hill", 'Pages':290, 'Format': "Ebook"}
                 , 7:{'Name':"Outwitting the devil: The secret to freedom and success", 'Author':"Napoleon Hill", 'Pages':302, 'Format': "Ebook"}
                 , 8:{'Name':"The four agreements", 'Author':"Don Miquel Ruiz", 'Pages':140, 'Format': "Print"}
                 , 9:{'Name':"Atomic habits: an easy and proven way to build good habits and break bad ones", 'Author':"James Clear", 'Pages':298, 'Format': "Print"}
                 , 10:{'Name':"How to win friends and influence people", 'Author':"Dale Carnegie", 'Pages':250, 'Format': "Print"}
                 , 11:{'Name':"Trading in the zone: master the market with confidence, discipline and a winning attitude", 'Author':"Mark Douglas", 'Pages':207, 'Format': "Print"}
                 , 12:{'Name':"Stone soup", 'Author':"Marcia Brown", 'Pages':52, 'Format': "Print"}
                 , 13:{'Name':"Rich dad's cashflow quadrant", 'Author':"Robert T. Kiyosaki", 'Pages':282, 'Format': "Ebook"}
                 , 14:{'Name':"A beginners guide to the stock market", 'Author':"Mathew R. Kratter", 'Pages':100, 'Format': "Ebook"}
                 , 15:{'Name':"Credit spread options for beginners: turn your most boring stocks into reliable monthly paychecks using call, put & iron butterfly spreads - even if the market is doing nothing", 'Author':"Freeman Publications", 'Pages':134, 'Format': "Ebook"}
                 , 16:{'Name':"Heart sick: Connecting the dots between hope, disappointment and healing", 'Author':"Conscious Core", 'Pages':107, 'Format': "Ebook"}
                 , 17:{'Name':"Thinking strategically: the competitive edge in business, politics and everyday life", 'Author':"Avinash K. Dixit & " "Barry J. Nalebuff", 'Pages':384, 'Format': "Ebook"}
                 , 18:{'Name':"Noise trading and illusory corrections in the US equity markets", 'Author':"Jennifer C. Bender & " "Carol L. Osler & " "David Simon", 'Pages':47, 'Format': "Online"}
                }

In [ ]:
BL21 = pd.DataFrame(book_list2021).transpose()
BL21

In [60]:
sum21 = pd.DataFrame(get_book_summaries(extract_delim(BL21['Name'])))

# show title with summaries
sum21.reset_index(drop=True, inplace=True)
sum21.style.set_properties(**{'text-align': 'left'}, inplace=True)
sum21

In [ ]:
pages_read21 = BL21['Pages'].sum()
print(f"Number of pages read in 2021: \t\t\t\t {format(pages_read21, ',d')} pages")
print(f"Average number of pages read per month in 2021: \t {round(pages_read21/12)} pages")
print(f"Average number of pages read per day in 2021: \t\t {round(pages_read21/days_in_year(2021))} pages")

In [ ]:
word_string=""

for book in BL21['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(
     background_color='black'
                       , scale=3, collocations=True
                       , colormap='tab20c'
                       , normalize_plurals=True
                       ).generate(word_string)

plt.figure(figsize = (20,6), facecolor = 'black')
plt.imshow(word_cloud, interpolation = 'bessel')
plt.axis('off')
plt.title("2021 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
book_list2022 = {1:{'Name':"The subtle art of not giving a F*ck", 'Author':"Mark Manson", 'Pages':206, 'Format': "Ebook"}
                 , 2:{'Name':"Why we sleep; unlocking the power of sleep and dreams", 'Author':"Mathew Walker", 'Pages':323, 'Format': "Ebook"}
                 , 3:{'Name':"Ingredients: the strange chemistry of what we put in and on us", 'Author':"Goerge Zaidan", 'Pages':299, 'Format': "Print"}
                 , 4:{'Name':"We should all be millionaires: A woman's guide to earning more, building wealth, and gaining economic power", 'Author':"Rachel Rodgers", 'Pages':276, 'Format': "Ebook"}
                 , 5:{'Name':"The five love languages", 'Author':"Gary Chapman", 'Pages':272, 'Format': "Audiobook"}
                 , 6:{'Name':"The art of saying NO", 'Author':"Damon Zahariades", 'Pages':160, 'Format': "Ebook"}
                 , 7:{'Name':"If you had controlling parents: How to make peace with your past and take your place in the world", 'Author':"Dan Neuharth", 'Pages':274, 'Format': "Ebook"}
                 , 8:{'Name':"Abundance beyond trauma: Discovering the courage for change and commitment to yourself", 'Author':"Jeannine L. Rashidi", 'Pages':169, 'Format': "Ebook"}
                 , 9:{'Name':"Dear Girl", 'Author':"Amy Krouse Rosenthal & " "Paris Rosenthal", 'Pages':40, 'Format': "Print"}
                }

In [ ]:
BL22 = pd.DataFrame(book_list2022).transpose()
BL22

In [ ]:
sum22 = pd.DataFrame(get_book_summaries(extract_delim(BL22['Name'])))

# show title with summaries
sum22.reset_index(drop=True, inplace=True)
sum22.style.set_properties(**{'text-align': 'left'}, inplace=True)
sum22

In [ ]:
pages_read22 = BL22['Pages'].sum()
print(f"Number of pages read in 2022: \t\t\t\t {format(pages_read22, ',d')} pages")
print(f"Average number of pages read per month in 2022: \t {round(pages_read22/12)} pages")
print(f"Average number of pages read per day in 2022: \t\t {round(pages_read22/days_in_year(2022))} pages")

In [ ]:
word_string=""

for book in BL22['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, normalize_plurals = True).generate_from_text(word_string)

plt.figure(figsize = (20,6), facecolor='black')
plt.imshow(word_cloud, interpolation = 'spline36')
plt.axis('off')
plt.title("2022 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
book_list2023 = {"Name":
    ["The 7 habits of highly effective people"
     ,"Why your weirdness is wonderful: Embrace your quirks and live your strengths"
     ,"Be Quiet!"
     ,"Red Riding Hood"
     ,"Storytelling with data a visualization guide for business professionals"
     ,"Learning SQL: Generating, Manipulating and Retrieving Data"]
                 , "Author":
    ["Stephen R. Covey"
    ,"Laurie Wallin"
    ,"Ryan T. Higgins"
    ,"Lydia L. Very"
    ,"Cole Nussbaumer Knaflic"
    ,"Alan Beaulieu"]
                 , "Pages":[556
                            , 193
                            , 22
                            , 53
                            , 255
                            , 610]
                 , "Format": ["Ebook"
                              , "Ebook"
                              , "Ebook"
                              , "Ebook"
                              , "Print"
                              , "Audiobook"]
                 }

In [ ]:
BL23 = pd.DataFrame(book_list2023)
BL23.index = BL23.index + 1
BL23

In [ ]:
sum23 = pd.DataFrame(get_book_summaries(extract_delim(BL23['Name'])))

# show title with summaries
sum23.reset_index(drop=True, inplace=True)
sum23.style.set_properties(**{'text-align': 'left'})
sum23

In [ ]:
pages_read23 = BL23['Pages'].sum()
print(f"Number of pages read in 2023: \t\t\t\t {format(pages_read23, ',d')} pages")
print(f"Average number of pages read per month in 2023: \t {round(pages_read23/12)} pages")
print(f"Average number of pages read per day in 2023: \t\t {round(pages_read23/days_in_year(2023))} pages")

In [ ]:
word_string=""

for book in BL23['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, normalize_plurals = True).generate_from_text(word_string)

plt.figure(figsize = (20,6), facecolor='black')
plt.imshow(word_cloud, interpolation = 'lanczos')
# 'antialiased', 'none', 'nearest', 'bilinear', 'bicubic', 'spline16', 'spline36', 'hanning', 'hamming', 'hermite', 'kaiser', 'quadric', 'catrom', 'gaussian', 'bessel', 'mitchell', 'sinc', 'lanczos', 'blackman'
plt.axis('off')
plt.title("2023 Reading List", fontweight='bold', color='black', backgroundcolor='gold')

plt.show()

In [ ]:
BL21['Year Read'] = 2021
BL22['Year Read'] = 2022
BL23['Year Read'] = 2023

In [ ]:
comb_list = pd.concat([BL21, BL22, BL23]).sort_values(by='Name').set_index([pd.Index(range(1, len(BL21) + len(BL22) + len(BL23) + 1))])
comb_list

In [ ]:
word_string=""

for book in comb_list['Name']:
     word_string += book.upper() + " "

word_cloud = WordCloud(background_color='black',scale=3, collocations=True, colormap='tab20c', normalize_plurals=True).generate_from_text(word_string)

plt.figure(figsize = (20,10), facecolor=(0,0,0,0.49))
plt.imshow(word_cloud)
plt.axis('off')
plt.title("Combined Reading List", fontweight='bold', color='gold', backgroundcolor='black', fontsize=15)

plt.show()

In [ ]:
print('Key Words in Book Titles')
comb_list[comb_list['Name'].str.contains(
    'power| abundance| heal| habit| time| invest| change| grow| trauma| discipline| courage'
    , case=False)]

In [ ]:
# Save file as CSV to create a power BI dashboard
labels = ['Short', 'Medium', 'Long']
bins = [0, 200, 280, 1000]
comb_list['Bin'] = pd.cut(comb_list['Pages'], bins=bins, labels=labels)

comb_list.sort_values(by=['Bin', 'Pages', 'Name'], ascending=True)
# comb_list.to_csv('reading_list.csv')